# WEEK 04 - Assigment

## Problem definition.

In [31]:
import folium
import numpy as np
import pandas as pd

from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
url = 'https://de.wikipedia.org/wiki/Stadtbezirke_M%C3%BCnchens'

df_munich = pd.read_html(url, index_col = 0)[0]
df_munich.reset_index(inplace = True)
df_munich.drop(columns = ['Nr.', 'Ausländer(%)'], inplace = True)
df_munich.columns = ['borough', 'area_km2', 'residents', 'density']
df_munich.head()

,borough,area_km2,residents,density
0,Altstadt-Lehel,315,21.126,6.716
1,Ludwigsvorstadt-Isarvorstadt,440,51.933,11.799
2,Maxvorstadt,430,51.834,12.060
3,Schwabing-West,436,68.935,15.800
4,Au-Haidhausen,422,61.654,14.611


In [3]:
df_munich.dtypes

borough       object
area_km2       int64
residents     object
density      float64
dtype: object

In [6]:
df_munich['residents'] = df_munich['residents'].apply(lambda x: x.replace('.', ''))
df_munich['residents'] = df_munich['residents'].apply(lambda x: int(x))
df_munich.head()

,borough,area_km2,residents,density
0,Altstadt-Lehel,315,21126,6.716
1,Ludwigsvorstadt-Isarvorstadt,440,51933,11.799
2,Maxvorstadt,430,51834,12.060
3,Schwabing-West,436,68935,15.800
4,Au-Haidhausen,422,61654,14.611


In [7]:
df_munich.dtypes

borough       object
area_km2       int64
residents      int64
density      float64
dtype: object

In [11]:
# Getting Munich coordinates

geoloc = Nominatim(user_agent = 'munich_explorer')
location = geoloc.geocode('Munich')
lat = location.latitude
long = location.longitude

print(f'Munich coordinates: {lat} ; {long}.')

Munich coordinate: 48.1371079 ; 11.5753822.


In [18]:
# Getting coordinates of each borough in Munich

tqdm.pandas()
geocode = RateLimiter(geoloc.geocode, min_delay_seconds=1)
coords = (df_munich['borough']).progress_apply(geocode)
coords

100%|██████████| 26/26 [00:25<00:00,  1.03it/s]


0     (Altstadt-Lehel, München, Bayern, Deutschland,...
1     (Ludwigsvorstadt-Isarvorstadt, München, Bayern...
2     (Maxvorstadt, München, Bayern, Deutschland, (4...
3     (Schwabing-West, München, Bayern, Deutschland,...
4     (Au-Haidhausen, München, Bayern, Deutschland, ...
5     (Sendling, München, Bayern, 81371, Deutschland...
6     (Sendling-Westpark, München, Bayern, Deutschla...
7     (Schwanthalerhöhe, Heimeranstraße, Bezirksteil...
8     (Neuhausen-Nymphenburg, München, Bayern, Deuts...
9     (Moosach, Glonn (VGem), Landkreis Ebersberg, B...
10    (Milbertshofen-Am Hart, München, Bayern, Deuts...
11    (Schwabing-Freimann, München, Bayern, Deutschl...
12    (Bogenhausen, Bezirksteil Englschalking, Bogen...
13    (Berg am Laim, München, Bayern, Deutschland, (...
14    (Trudering-Riem, München, Bayern, Deutschland,...
15    (Ramersdorf-Perlach, München, Bayern, Deutschl...
16    (Obergiesing-Fasangarten, München, Bayern, Deu...
17    (Untergiesing-Harlaching, München, Bayern,

In [21]:
coords[0]

Location(Altstadt-Lehel, München, Bayern, Deutschland, (48.1378285, 11.5745823, 0.0))

In [24]:
# Creating latitude and longitude features.

df_munich['lat'] = np.nan
df_munich['long'] = np.nan

for i in df_munich.index:
    df_munich.at[i, 'lat'] = coords[i].latitude
    df_munich.at[i, 'long'] = coords[i].longitude

df_munich.head()

,borough,area_km2,residents,density,lat,long
0,Altstadt-Lehel,315,21126,6.716,48.137828,11.574582
1,Ludwigsvorstadt-Isarvorstadt,440,51933,11.799,48.130340,11.573366
2,Maxvorstadt,430,51834,12.060,48.151092,11.562418
3,Schwabing-West,436,68935,15.800,48.168271,11.569873
4,Au-Haidhausen,422,61654,14.611,48.128753,11.590536


In [30]:
# Including 'search_radius' column to be used on Foursqaure API searchs.

df_munich['search_radius'] = df_munich['area_km2'].apply(lambda x: round(np.sqrt(x / np.pi) * 1000))
df_munich.head()

,borough,area_km2,residents,density,lat,long,search_radius
0,Altstadt-Lehel,315,21126,6.716,48.137828,11.574582,10013.0
1,Ludwigsvorstadt-Isarvorstadt,440,51933,11.799,48.130340,11.573366,11835.0
2,Maxvorstadt,430,51834,12.060,48.151092,11.562418,11699.0
3,Schwabing-West,436,68935,15.800,48.168271,11.569873,11781.0
4,Au-Haidhausen,422,61654,14.611,48.128753,11.590536,11590.0


**After get the Munich map done for the first time, it was observed an "outlier". the "Moosach" borough has the same name as a german city and the geocode bring the city coordinates. So, the Moosach borough real coordinates are: 48.183333, 11.516667**

In [66]:
df_munich.loc[9] = ['Moosach', 1109, 54872, 4.946, 48.183333, 11.516667, 18788.0]
df_munich

,borough,area_km2,residents,density,lat,long,search_radius
0,Altstadt-Lehel,315,21126,6.716,48.137828,11.574582,10013.0
1,Ludwigsvorstadt-Isarvorstadt,440,51933,11.799,48.130340,11.573366,11835.0
2,Maxvorstadt,430,51834,12.060,48.151092,11.562418,11699.0
3,Schwabing-West,436,68935,15.800,48.168271,11.569873,11781.0
4,Au-Haidhausen,422,61654,14.611,48.128753,11.590536,11590.0
5,Sendling,394,41256,10.475,48.118012,11.539083,11199.0
6,Sendling-Westpark,781,60498,7.742,48.118031,11.519333,15767.0
7,Schwanthalerhöhe,207,29611,14.303,48.133782,11.541057,8117.0
8,Neuhausen-Nymphenburg,1291,100213,7.760,48.154222,11.531517,20272.0
9,Moosach,1109,54872,4.946,48.183333,11.516667,18788.0


In [67]:
# Munich map visualization

munich_map = folium.Map(location = [lat, long], zoom_start = 10)

for lat, long, borough, residents in zip(df_munich['lat'],
                                         df_munich['long'],
                                         df_munich['borough'], 
                                         df_munich['residents']):
    label = f'{borough} - Pop: {residents}'
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, long],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = 'darkblue',
        fill_opacity = 0.7,
        parse_html = False).add_to(munich_map)

munich_map    